### Acquisition and sampling

Our surrogate method relies on machine learning model's predictions of randomly sampled data around promising points. This random data is sampled from truncated normal distributions which standard deviation gets narrower with each optimization cycle $t$.
In this tutorial, we will also see how the exploration degree matters in the described endevour.  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
plt.style.use("seaborn-v0_8-paper")

font = 16
plt.rcParams.update({
    'text.usetex': False,
    'font.family': 'arial',
    'font.size': font,
    'axes.labelsize': font,  
    'xtick.labelsize': font,  
    'ytick.labelsize': font, 
    'legend.fontsize': font,
    'legend.title_fontsize': font,
    'axes.titlesize': font
})
import warnings
warnings.filterwarnings("ignore")

### Exploration vs. Exploitation 

Each optimization cycle decreases the standard deviation of normal distributions $\mathcal{N}_{\mathrm{trunc}}(\mu_{p}, \sigma_{p}(t))$ around each parameter value $\mu_{p}=x_p$. This is done based on the current cycle $t$ of the optimization process. Thereby, the focus of discovering new, but less favorable configurations (exploration) gradually shifts towards refining known and well performing configurations (exploitation). To this end, the distribution is truncated at $x_p^\mathrm{min}$ and $x_p^\mathrm{max}$ with standard deviation $\sigma_p(t) = \gamma(t) \cdot (x_p^{\mathrm{max}}-x_p^\mathrm{min})/2$, where $\gamma(t)$ is a monotonically decreasing function for $0\le t\le T$ (see Proof \ref{proof:gamma} below). Initially, $\sigma_p(t=0) = (x_p^{\mathrm{max}}-x_p^\mathrm{min})/2$ and it gradually narrows as $\gamma(t) > \gamma(t+1)$. We use the smooth function
\begin{align}
    \gamma(t) = (1-\ln(1+t/T)^2)^d,\ \text{where } d\ge 1
\end{align} 
for this purpose, where $T$ is the maximum number of optimization cycles and $d$ is chosen according to the degree of exploitation desired; 

In [ ]:
T = 1
gamma = lambda count, d: (1-np.log(1+count/int(T))**2)**d
x = np.linspace(0,T,100)
for i,d in enumerate(range(0,8,2)):
    plt.plot(x, [gamma(count,d) for count in x], label=f'd={d}', color=sns.color_palette()[i], ls=['-', '--', '-.', ':'][i])
plt.legend()
plt.ylabel(r'$\gamma(t)$')
plt.xlabel(r'$t/T$')
plt.grid()
plt.tight_layout()
plt.savefig('transition_function.pdf')

Assume $d=4$ and we just started the surrogate optimization process $t=0$

In [ ]:
d = 2
gamma(0,d)

This means, that we sample with the full standard deviation.

How does the truncated normal distribution (from which we sample the points for the machine learning models) look like?

In [ ]:
from scipy.stats import truncnorm

xmin = 2
xmax = 7
x = 4
std = gamma(0,d) * (xmax - xmin)/2
sample = truncnorm.rvs((xmin - x) / std, (xmax+1 - x) / std, loc=x, scale=std, size=10000)
sample = sample.astype(int)

In [ ]:
fig = plt.hist(sample, bins=50)
plt.xlim((0,9))
plt.show()

Now assume $d=4$ and the end of the optimization process is reached $t=T$.

In [ ]:
d=2
gamma(1,d)

In [ ]:
std = gamma(1,d) * (xmax - xmin)/2
sample = truncnorm.rvs((xmin - x) / std, (xmax+1 - x) / std, loc=x, scale=std, size=10000)
sample = sample.astype(int)

In [ ]:
fig = plt.hist(sample, bins=50)
plt.xlim((0,9))
plt.show()

In [1]:
a = [0,1,2,3,4,5]

In [4]:
a[::-2]

[5, 3, 1]

As expected, at $t=T$ sampling takes place much closer to the mean (current parameter value $x_p$).